In [6]:
import sys
sys.path.append("../src/")
sys.path.append("../")



import torch
from bert import *
from feature_data import *
from multiprototype import *
from models import *
from utils import *
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

In [7]:
bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpnl1m62mf
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

Load in csv of words and ontexts collected from corpora

In [3]:
"""
take a look at the eval words
"""
pd.set_option('display.max_colwidth', None)
eval_words = pd.read_csv('../data/processed/feature_annotation_pilot_annotation_items.csv')
eval_words

,sentenceid,word,literal?,pos,construction,sentence,corpus,info,genre,source,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1,collapse,literal,n,ADJ X,"The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods.",coca,COCA:2012:BLOG,yankeeanalysts.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,collapse,literal,n,colapse of X,Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.,coca,COCA:2012:BLOG,freethoughtblogs.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,collapse,literal,n,collapse of X,"Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen.",coca,2012,BLOG,...square.blogs.cnn.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,collapse,literal,n,collapse of X,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,coca,1998,MAG,Cosmopolitan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,collapse,literal,n,NOUN collapse,"After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated outside. The safest place to sleep is out in the open.",coca,2018,SPOK,PBS_Newshour,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,collapse,literal,n,NOUN collapse,"By now, everybody has heard about the plight of 33 men trapped deep underground in Chile by a mine collapse and that they're going to have to wait for months yet before rescue.",coca,2010,SPOK,NPR_TalkNation,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,collapse,literal,v,collapse PREP,"Veins bulging and throbbing, he struggled for breath and collapsed on the floor.",coca,FIC: Goddess of fire,2017,Bharti Kirchner,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,collapse,literal,v,collapse PREP,Roads may have weakened and could collapse under the weight of a car.,coca,COCA:2012:WEB,ready.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,collapse,literal,v,collapse PREP,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.",coca,COCA:2003:FIC,MassachRev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,collapse,metaphorical,n,collapse of X,"The collapse of the Mobutu dictatorship led to a collision between two forms of opposition to Mobutu, the internal opposition exemplified by Tshisekedi, committed to non-violent direct action, and the armed opposition, based abroad, exemplified by Kabila.",mohler_2015,05 Engineering and Business: Engineering,Built,dictatorship,NaN,collapse,dictatorship,Government,collapse,Building,7.213e+05


In [4]:
"""
models and save paths

"""
buchanan_models = [
    '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters',
    '../trained_models/model.plsr.buchanan.allbuthomonyms.1k.300components.500max_iters',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.modabs.buchanan.allbuthomoyms.5k',
    #'../trained_models/model.modabs.buchanan.allbuthomoyms.1k',
]

buchanan_glove_models = [
    '../trained_models/model.plsr.buchanan.allbuthomonyms.glove.300components.300max_iters',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.modabs.buchanan.allbuthomoyms.glove'   
]

mcrae_models = [
    '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.mc_rae_real.allbuthomonyms.5k.100components.500max_iters',
    '../trained_models/model.plsr.mc_rae_real.allbuthomonyms.1k.50components.500max_iters',
    '../trained_models/model.modabs.mc_rae_real.5k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
    '../trained_models/model.modabs.mc_rae_real.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4',
]

mcrae_glove_models = [
    #'../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.plsr.mc_rae_real.allbuthomonyms.glove.100components.500max_iters',
    #'../trained_models/model.modabs.mc_rae_real.glove.mu1_1.mu2_0.1.mu3_1e-07.mu4_5.nnk_5',
]

In [13]:


"""
data is a list of tuples of (cue word, lemma, context)

for each model, we want to run an analysis of these
"""

def pluralize(string):
    if string == 'mouse':
        return 'mice'
    else:
        return string + 's'

    
def evaluate_in_context(model, df, bert, glove=False, model_name=None):
    """
    df has columns
    cue_word
    lemma
    context
    """
    n = 0
    
    cosines = []
    top_10_precs = []
    top_20_precs = []
    top_k_precs = []
    correlations = []
    rows = []

    num_top_10 = 0
    num_top_20 = 0
    num_total = 0

    for index, row in df.iterrows():
        n +=1

        # predict features in context
        singular = row.word
        plural = pluralize(singular)
        past = singular+"d"
        try:
            prediction = model.predict_in_context(singular, row.sentence, bert)
            word = singular
        except:
            try:
                prediction = model.predict_in_context(plural, row.sentence, bert)
                word = plural
            except:
                prediction = model.predict_in_context(past, row.sentence, bert)
                word = past

#         # get gold feature
#         gold_vector = model.feature_norms.get_feature_vector(cue_word)
#         gold_feats = model.feature_norms.get_features(cue_word)
#         k = len(gold_feats)

#         # get cosines
#         cos = 1 - cosine(prediction, gold_vector)
#         cosines.append(cos)

#         # get MAP at 10
        top_10 = model.predict_top_n_features_in_context(row.word, row.sentence, 10, vec=prediction, bert=bert)
#         top_10_gold = model.feature_norms.top_n(cue_word, 10)
        
#         num_in_top_10 = len(set(top_10).intersection(set(top_10_gold)))
#         top_10_prec = num_in_top_10 / len(top_10_gold)
#         top_10_precs.append(top_10_prec)

#         # get MAP at 20
#         top_20 = model.predict_top_n_features_in_context(word, context, 10, vec=prediction, bert=bert)
#         top_20_gold = model.feature_norms.top_n(cue_word, 20)

#         num_in_top_20 = len(set(top_20).intersection(set(top_20_gold)))
#         top_20_prec = num_in_top_20 / len(top_20_gold)
#         top_20_precs.append(top_20_prec)

#         # get MAP at k
#         gold_len = len(gold_feats)
#         top_k = model.predict_top_n_features_in_context(word, context, gold_len, vec=prediction, bert=bert)
#         num_in_top_k = len(set(top_k).intersection(set(gold_feats)))
#         top_k_prec = num_in_top_k / gold_len
#         top_k_precs.append(top_k_prec)

#         # get correlation
#         corr, p = spearmanr(prediction, gold_vector)
#         correlations.append(corr)

        #if ((i % 20 ==0) and debug=='info') or (debug=='true'):
#         if False:
#             print(word)
#             print("top ten predicted features: ", top_10)
#             print("top ten gold features: ", top_10_gold)
#             print("gold features: ", gold_feats)
#             print("top k predicted features: ", top_k)

#             print("cosine: %f" % cos)
#             print("correlation: %f" % corr)
#             print("top k prec: %f" % top_k_prec)
    
    
#        row = (cue_word, context, top_10_prec, top_20_prec, top_k_prec, top_10, top_10_gold, top_k, gold_feats, cos)
        row = (row.word, row.sentence, top_10)
        rows.append(row)
    
    cols = ['word', 
            'sentence', 
#                'lemma', 
#                'top_10_prec', 
#                'top_20_prec', 
#                'top_k_prec',
            'top_10', 
#                'top_10_gold', 
#                'top_k', 
#                'gold_feats', 
#                'cos'
           ]
    
        
    avg_top_10_prec = np.average(top_10_precs)
    avg_top_20_prec = np.average(top_20_precs)
    avg_top_k_prec = np.average(top_k_precs)
    average_correlation = np.average(correlations)
    average_cosine = np.average(cosines)
        
        
#     print("Average cosine between gold and predicted feature norms: %s" % average_cosine)
#     print("average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector: ", top_10_prec)
#     print("average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector: ", top_20_prec)
#     print("Average % @k (derby metric)", top_k_prec)
#     print("correlation between gold and predicted vectors: %s " % average_correlation)        
#     print("total number of predictions: ", n)
    
    results_df = pd.DataFrame.from_records(rows, columns = cols)

    results_df['model'] = model_name

    
    return results_df
    



# Run on McRae Models

In [14]:
# then we make predictions for each word

dfs = []

for save_path in mcrae_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)

    df = evaluate_in_context(model, eval_words, bert, model_name=save_path)
    dfs.append(df)
    
# need to do this separately because we dont have nice self examining attributes for our model classes to tell when its 
# been trained on glove
for save_path in mcrae_glove_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, glove=True, model_name=save_path)
    dfs.append(df)

df = pd.concat(dfs)
df.to_csv('../results/feature_annotation_pilot_annotation_items_metaphor_predictions.csv')

****************************************
*** Evaluating ../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300 model ***
****************************************
collapse
The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods. 
collapse
Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.
collapse
 Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen. 
collapse
Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments. 
col

flow
Place a large rimmed pan on the rack underneath the rack that holds the turkey to catch the drippings. The rack is for air flow; it will help your turkey cook more evenly.
flow
When you smoke, blood does not properly flow throughout your body and the recovery time from the surgery will increase.
flow
Then, when energy demands exceeded generating capacity, water would be allowed to flow east of the Cascades, with the electricity generated would be used to meet demand, and the water would be used locally for irrigation and other uses, and what was n't used right away would be stored for future use
flow
The black areas show space for scent to flow into the patient's nose, this becomes blocked in patients suffering with chronic sinusitis
flow
They forget what can happen when they allow the free flow of firearms into public places, especially forms of transportation.
flow
Most experts believe that increased criminal penalties of the kind proposed in the federal law could help play a ro

cure
I agree with Litman's implication that poverty is a cure for congestion, but I don't think it is an appropriate one.
cure
There is a tendency to think that wealth is a cure for one's barbaric and primitive tendencies when it merely just mask it from the outside world.
cure 
The Arab Spring arose from the prescription of political reform as the democracy cure for what ails the Muslim world.
cure 
The same system which creates poverty can not cure it.
cure 
Democracy may cure the symptoms, but it will not cure what really ails the muslim world.
cure 
Multiply Huxley's formula by countless politicians endlessly reciting "low taxes create economic growth," in innumerable elections for offices at every level for the last 30 years, supported in turn by a legion of corporately sponsored think tanks cranking out selective  "documentation" , and we have the Received Wisdom of 21st century American politics: Low taxes cure anything that might ail us.
flow
Destructo got used to bottles when 

cure
She's committed to helping raise money to find a cure for breast cancer, the disease that took her grandmother's life (Avon's donated $350 million and mammogram equipment to countries like Brazil). 
cure
Farber, in trying to find a cure for his wife's blindness, has created a device that allows the user to send images directly to the brain, enabling the blind to see. 
cure
Have you forgotten our pact? Marry me, and I'll provide the cure for the poison that's killing him. Then I'll get to work slaughtering every last puny mortal who opposes me.
cure
Homoeopathic cure of natural diseases relates to giving the most expedient remedy which has been proved on healthy individuals for their pure action.
cure
 ... but the Psychiatrist tells us Floyd, Jr. will never fight Pacquiao because the virus remains in the inner center of his nervous center... all allied medical professionals, therapist, psycologist, psychiatrist, opthalmologist, neurologist, dentist, physician, entomogist, geologist

/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

flow
When you smoke, blood does not properly flow throughout your body and the recovery time from the surgery will increase.
flow
Then, when energy demands exceeded generating capacity, water would be allowed to flow east of the Cascades, with the electricity generated would be used to meet demand, and the water would be used locally for irrigation and other uses, and what was n't used right away would be stored for future use
flow
The black areas show space for scent to flow into the patient's nose, this becomes blocked in patients suffering with chronic sinusitis
flow
They forget what can happen when they allow the free flow of firearms into public places, especially forms of transportation.
flow
Most experts believe that increased criminal penalties of the kind proposed in the federal law could help play a role in stanching the flow of guns from traffickers to criminals.
flow
There will be a much larger flow of taxes going into the governmental coffers.
flow
Distance and Open Learni

/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


collapse
The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods. 
collapse
Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.
collapse
 Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen. 
collapse
Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments. 
collapse
 After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated 

flow
When you smoke, blood does not properly flow throughout your body and the recovery time from the surgery will increase.
flow
Then, when energy demands exceeded generating capacity, water would be allowed to flow east of the Cascades, with the electricity generated would be used to meet demand, and the water would be used locally for irrigation and other uses, and what was n't used right away would be stored for future use
flow
The black areas show space for scent to flow into the patient's nose, this becomes blocked in patients suffering with chronic sinusitis
flow
They forget what can happen when they allow the free flow of firearms into public places, especially forms of transportation.
flow
Most experts believe that increased criminal penalties of the kind proposed in the federal law could help play a role in stanching the flow of guns from traffickers to criminals.
flow
There will be a much larger flow of taxes going into the governmental coffers.
flow
Distance and Open Learni

cure
I agree with Litman's implication that poverty is a cure for congestion, but I don't think it is an appropriate one.
cure
There is a tendency to think that wealth is a cure for one's barbaric and primitive tendencies when it merely just mask it from the outside world.
cure 
The Arab Spring arose from the prescription of political reform as the democracy cure for what ails the Muslim world.
cure 
The same system which creates poverty can not cure it.
cure 
Democracy may cure the symptoms, but it will not cure what really ails the muslim world.
cure 
Multiply Huxley's formula by countless politicians endlessly reciting "low taxes create economic growth," in innumerable elections for offices at every level for the last 30 years, supported in turn by a legion of corporately sponsored think tanks cranking out selective  "documentation" , and we have the Received Wisdom of 21st century American politics: Low taxes cure anything that might ail us.
flow
Destructo got used to bottles when 

In [16]:
df

,word,sentence,top_10,model
0,collapse,"The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods.","[has_a_tail, has_legs, is_small, is_hard, is_brown, made_of_metal, has_4_legs, a_tool, an_animal, is_large]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
1,collapse,Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.,"[is_heavy, has_a_beak, made_of_metal, a_tool, beh_-_flies, has_feathers, has_wings, a_bird, an_animal, is_large]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
2,collapse,"Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen.","[is_heavy, is_tall, a_bird, an_animal, is_round, a_weapon, a_tool, made_of_metal, is_hard, is_large]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
3,collapse,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,"[is_dangerous, used_for_carpentry, is_large, is_sharp, is_hard, an_animal, is_heavy, has_a_handle, made_of_metal, a_tool]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
4,collapse,"After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated outside. The safest place to sleep is out in the open.","[beh_-_flies, is_hard, used_for_war, is_dangerous, is_large, a_tool, a_weapon, an_animal, made_of_metal, used_for_killing]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
5,collapse,"By now, everybody has heard about the plight of 33 men trapped deep underground in Chile by a mine collapse and that they're going to have to wait for months yet before rescue.","[used_for_war, is_hard, made_of_metal, an_animal, is_large, a_weapon, used_for_killing, is_heavy, a_tool, is_dangerous]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
6,collapse,"Veins bulging and throbbing, he struggled for breath and collapsed on the floor.","[used_for_sleeping, has_legs, has_a_tail, is_soft, used_for_killing, an_animal, a_weapon, is_comfortable, has_4_legs, is_brown]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
7,collapse,Roads may have weakened and could collapse under the weight of a car.,"[used_for_construction, a_weapon, is_brown, a_fruit, an_animal, is_heavy, a_tool, is_hard, made_of_metal, is_large]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
8,collapse,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.","[has_fur, has_a_tail, is_large, made_of_metal, has_4_legs, has_legs, is_hard, is_round, is_brown, an_animal]",../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
9,collapse,"The collapse of the Mobutu dictatorship led to a collision between two forms of opposition to Mobutu, the internal opposition exemplified by Tshisekedi, committed to non-violent direct action, and the armed opposition, based abroad, exemplified by Kabila.","[beh_-_lays_eggs, used_for_killing, a_weapon, is_dangerous, has_wings, has_a_beak, has_feathers, beh_-_flies, a_bird, is_large]",../trained_models/model.ffnn.mc_rae_real.allbuthomonym

# Run on Buchanan Models

In [33]:
dfs = []

for save_path in buchanan_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, model_name=save_path)
    dfs.append(df)

# for save_path in buchanan_glove_models:
#     print("****************************************")
#     print("*** Evaluating %s model ***" % save_path)
#     print("****************************************")
#     model = torch.load(save_path)
#     df = evaluate_in_context(model, eval_words, bert, glove=True, model_name=save_path)
#     dfs.append(df)


df = pd.concat(dfs)
df.to_csv('../results/feature_annotation_pilot_annotation_items_metaphor_predictions_buchanan.csv')

****************************************
*** Evaluating ../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters model ***
****************************************
collapse
The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods. 
collapse
Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.
collapse
 Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen. 
collapse
Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments. 
collapse
 After

flow
When you smoke, blood does not properly flow throughout your body and the recovery time from the surgery will increase.
flow
Then, when energy demands exceeded generating capacity, water would be allowed to flow east of the Cascades, with the electricity generated would be used to meet demand, and the water would be used locally for irrigation and other uses, and what was n't used right away would be stored for future use
flow
The black areas show space for scent to flow into the patient's nose, this becomes blocked in patients suffering with chronic sinusitis
flow
They forget what can happen when they allow the free flow of firearms into public places, especially forms of transportation.
flow
Most experts believe that increased criminal penalties of the kind proposed in the federal law could help play a role in stanching the flow of guns from traffickers to criminals.
flow
There will be a much larger flow of taxes going into the governmental coffers.
flow
Distance and Open Learni

/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


collapse
The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods. 
collapse
Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.
collapse
 Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen. 
collapse
Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments. 
collapse
 After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated 

flow
When you smoke, blood does not properly flow throughout your body and the recovery time from the surgery will increase.
flow
Then, when energy demands exceeded generating capacity, water would be allowed to flow east of the Cascades, with the electricity generated would be used to meet demand, and the water would be used locally for irrigation and other uses, and what was n't used right away would be stored for future use
flow
The black areas show space for scent to flow into the patient's nose, this becomes blocked in patients suffering with chronic sinusitis
flow
They forget what can happen when they allow the free flow of firearms into public places, especially forms of transportation.
flow
Most experts believe that increased criminal penalties of the kind proposed in the federal law could help play a role in stanching the flow of guns from traffickers to criminals.
flow
There will be a much larger flow of taxes going into the governmental coffers.
flow
Distance and Open Learni

/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


collapse
The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods. 
collapse
Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.
collapse
 Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen. 
collapse
Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments. 
collapse
 After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated 

flow
When you smoke, blood does not properly flow throughout your body and the recovery time from the surgery will increase.
flow
Then, when energy demands exceeded generating capacity, water would be allowed to flow east of the Cascades, with the electricity generated would be used to meet demand, and the water would be used locally for irrigation and other uses, and what was n't used right away would be stored for future use
flow
The black areas show space for scent to flow into the patient's nose, this becomes blocked in patients suffering with chronic sinusitis
flow
They forget what can happen when they allow the free flow of firearms into public places, especially forms of transportation.
flow
Most experts believe that increased criminal penalties of the kind proposed in the federal law could help play a role in stanching the flow of guns from traffickers to criminals.
flow
There will be a much larger flow of taxes going into the governmental coffers.
flow
Distance and Open Learni

cure
I agree with Litman's implication that poverty is a cure for congestion, but I don't think it is an appropriate one.
cure
There is a tendency to think that wealth is a cure for one's barbaric and primitive tendencies when it merely just mask it from the outside world.
cure 
The Arab Spring arose from the prescription of political reform as the democracy cure for what ails the Muslim world.
cure 
The same system which creates poverty can not cure it.
cure 
Democracy may cure the symptoms, but it will not cure what really ails the muslim world.
cure 
Multiply Huxley's formula by countless politicians endlessly reciting "low taxes create economic growth," in innumerable elections for offices at every level for the last 30 years, supported in turn by a legion of corporately sponsored think tanks cranking out selective  "documentation" , and we have the Received Wisdom of 21st century American politics: Low taxes cure anything that might ail us.
flow
Destructo got used to bottles when 

In [34]:
df

,word,sentence,top_10,model
0,collapse,"The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods.","[intent, up, human, hurt, down, fall, body, break, act, move]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
1,collapse,Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.,"[fall, hurt, force, destroy, down, object, human, act, break, move]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
2,collapse,"Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen.","[physic, build, bad, object, act, break, hurt, human, destroy, move]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
3,collapse,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,"[intent, hurt, fall, down, human, body, break, up, move, act]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
4,collapse,"After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated outside. The safest place to sleep is out in the open.","[force, fall, noise, move, hurt, object, act, destroy, break, human]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
5,collapse,"By now, everybody has heard about the plight of 33 men trapped deep underground in Chile by a mine collapse and that they're going to have to wait for months yet before rescue.","[destroy, force, object, human, fall, person, break, act, move, hurt]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
6,collapse,"Veins bulging and throbbing, he struggled for breath and collapsed on the floor.","[body, sleep, human, person, hurt, up, break, bad, fall, down]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
7,collapse,Roads may have weakened and could collapse under the weight of a car.,"[force, voluntary, down, up, object, human, hurt, break, act, fall]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
8,collapse,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.","[intent, leg, voluntary, up, hurt, act, human, fall, break, animal]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters
9,collapse,"The collapse of the Mobutu dictatorship led to a collision between two forms of opposition to Mobutu, the internal opposition exemplified by Tshisekedi, committed to non-violent direct action, and the armed opposition, based abroad, exemplified by Kabila.","[force, destroy, object, noise, loud, danger, move, human, end, break]",../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters


# Run on Binder Models

In [28]:
# TODO hyperparameter tuning for binder models
# TODO train on all binder words not just train split

binder_models = [
    '../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.binder.5k.300components.500max_iters',
    '../trained_models/model.plsr.binder.1k.300components.500max_iters',
    '../trained_models/model.modabs.binder.5k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
    '../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
]

binder_glove_models = [
    '../trained_models/model.ffnn.binder.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.binder.glove.300components.500max_iters',
    '../trained_models/model.modabs.binder.glove.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
]

In [71]:
dfs = []

for save_path in binder_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, model_name=save_path)
    dfs.append(df)

# for save_path in binder_glove_models:
#     print("****************************************")
#     print("*** Evaluating %s model ***" % save_path)
#     print("****************************************")
#     model = torch.load(save_path)
#     df = evaluate_in_context(model, eval_words, bert, glove=True, model_name=save_path)
#     dfs.append(df)


df = pd.concat(dfs)
df.to_csv('../results/feature_annotation_pilot_annotation_items_metaphor_predictions_binder.csv')

****************************************
*** Evaluating ../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300 model ***
****************************************


AttributeError: 'Series' object has no attribute 'word'

In [31]:
df

,word,sentence,top_10,model
0,collapse,"The team did not procure a contrast MRI, before the trade, the most accurate image to detect small labrum the second half of the 2011 season, his quick collapse during early spring training and lack of velocity early on should make us allwonder if the Yankees did in fact receive damaged goods.","[Surprised, Fearful, Angry, Attention, Pain, Harm, Unpleasant, Sad, Consequential, Arousal]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
1,collapse,Other leading ice scientists this week predicted the complete collapse of sea ice in the Arctic within four years.,"[Pain, Drive, Loud, Surprised, Fearful, Arousal, Consequential, Unpleasant, Attention, Harm]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
2,collapse,"Was that verified over night, or did we suffer days and months of visions of the collapse of the towers, as well as national introspection on how this could ever happen.","[Vision, Sad, Angry, Unpleasant, Fearful, Harm, Consequential, Pain, Attention, Arousal]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
3,collapse,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,"[Surprised, Fearful, Angry, Pain, Sad, Arousal, Harm, Attention, Consequential, Unpleasant]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
4,collapse,"After nightfall, on the other side of the island, people who were injured in the quake made it to hospital. But the fear of aftershocks and another building collapse meant that they were treated outside. The safest place to sleep is out in the open.","[Sad, Loud, Surprised, Arousal, Pain, Fearful, Consequential, Harm, Attention, Unpleasant]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
5,collapse,"By now, everybody has heard about the plight of 33 men trapped deep underground in Chile by a mine collapse and that they're going to have to wait for months yet before rescue.","[Sad, Loud, Pain, Arousal, Surprised, Fearful, Consequential, Harm, Attention, Unpleasant]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
6,collapse,"Veins bulging and throbbing, he struggled for breath and collapsed on the floor.","[Short, Attention, Arousal, UpperLimb, Unpleasant, Harm, Biomotion, Pain, Body, Consequential]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
7,collapse,Roads may have weakened and could collapse under the weight of a car.,"[Loud, Shape, Surprised, Fearful, Harm, Pain, Consequential, Large, Attention, Unpleasant]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
8,collapse,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.","[Short, Sad, Arousal, Pain, Fearful, Surprised, Unpleasant, Harm, Attention, Consequential]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
9,collapse,"The collapse of the Mobutu dictatorship led to a collision between two forms of opposition to Mobutu, the internal opposition exemplified by Tshisekedi, committed to non-violent direct action, and the armed opposition, based abroad, exemplified by Kabila.","[Angry, Sad, Surprised, Social, Fearful, Unpleasant, Harm, Consequential, Attention, Arousal]",../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300


# Load our own feature annotations for comparison

In [11]:
#load gold feature norms:

"""
take a look at the gold features
"""
pd.set_option('display.max_colwidth', None)
annot = pd.read_csv("../data/raw/Initial Binder Annotation Form (Responses) - Copy of Form Responses 1.csv")
annot

,Timestamp,Email Address,SentenceID,SentenceText,Features [Vision - Vision],Features [Vision - Bright],Features [Vision - Dark],Features [Vision - Color],Features [Vision - Pattern],Features [Vision - Large],Features [Vision - Small],Features [Vision - Motion],Features [Vision - Biomotion],Features [Vision - Fast],Features [Vision - Slow],Features [Vision - Shape],Features [Vision - Complexity],Features [Vision - Face],Features [Vision - Body],Features [Somatic - Touch],Features [Somatic - Temperature],Features [Somatic - Texture],Features [Somatic - Weight],Features [Somatic - Pain],Features [Audition - Audition],Features [Audition - Loud],Features [Audition - Low],Features [Audition - High],Features [Audition - Sound],Features [Audition - Music],Features [Audition - Speech],Features [Gustation - Taste],Features [Olfaction - Smell],Features [Motor - Head],Features [Motor - UpperLimb],Features [Motor - LowerLimb],Features [Motor - Practice],Features [Spatial - Landmark],Features [Spatial - Path],Features [Spatial - Scene],Features [Spatial - Near],Features [Spatial - Toward],Features [Spatial - Away],Features [Spatial - Number],Features [Temporal - Time],Features [Temporal - Duration],Features [Temporal - Long],Features [Temporal - Short],Features [Causal - Caused],Features [Causal - Consequential],Features [Social - Social],Features [Social - Human],Features [Social - Communication],Features [Social - Self],Features [Cognition - Cognition],Features [Emotion - Benefit],Features [Emotion - Harm],Features [Emotion - Pleasant],Features [Emotion - Unpleasant],Features [Emotion - Happy],Features [Emotion - Sad],Features [Emotion - Angry],Features [Emotion - Disgusted],Features [Emotion - Fearful],Features [Emotion - Surprised],Features [Drive - Drive],Features [Drive - Needs],Features [Attention - Attention],Features [Attention - Arousal]
0,3/9/2022 17:22:31,katrin.erk@utexas.edu,4,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,1,0,3,3,0,0,1,3,3,0,1,6,4,0,6,3,3,3,0,6,0,0,0,0,0,0,0,0,3,0,3,3,0,3,5,1,5,4,4,0,0,0,4,0,4,6,0,5,4,3,4,0,6,0,6,0,6,3,1,6,4,4,6,4,4
1,3/21/2022 14:05:20,gabriellachronis@utexas.edu,4,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,1,0,4,3,0,0,4,1,0,0,3,3,4,0,5,0,1,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,3,1,1,0,0,6,6,0,1,0,0,1,0,6,0,6,0,2,2,4,6,4,2,2,0,3
2,3/10/2022 19:33:15,katrin.erk@utexas.edu,9,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.",5,0,0,2,0,3,3,4,4,3,3,6,0,3,6,0,0,0,3,5,1,0,0,0,2,0,0,0,0,2,3,6,0,3,4,6,3,5,0,0,3,3,1,2,2,5,3,6,1,0,4,0,5,0,3,0,3,0,1,3,0,0,0,4,1
3,3/21/2022 14:25:53,gabriellachronis@utexas.edu,9,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.",6,2,4,0,0,0,2,6,6,5,1,0,0,1,6,1,0,0,0,0,4,5,1,0,0,1,0,0,0,2,3,6,3,0,6,4,0,3,0,0,6,6,0,4,6,2,3,3,0,1,0,0,2,1,5,0,0,0,0,2,6,3,0,3,5
4,3/9/2022 20:21:36,katrin.erk@utexas.edu,13,"Over the past quarter century, the world has witnessed the fall of communism in Eastern Europe, the end of apartheid in South Africa, and the collapse of military dictatorships in Latin America.",0,0,0,0,0,2,0,2,2,3,0,0,0,3,3,0,0,0,0,2,3,3,0,0,0,1,3,0,0,1,1,1,0,0,0,0,0,0,0,0,0,3,3,0,6,6,6,6,6,1,2,6,3,6,3,6,3,5,5,5,5,5,1,6,6
5,3/21/2022 14:16:25,gabriellachronis@utexas.edu,13,"Over the past quarter century, the world has witnessed the fall of communism in Eastern Europe, the end of apartheid in South Africa, and the collapse of military dictatorships in Latin America.",3,0,0,0,0,3,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,2,0,6,0,1,0,0,5,3,2,0,5,6,6,2,0,0,5,3,4,0,4,3,3,4,5,0,0,4,

In [12]:
"""
apparently the binder data we got from the internet doesnt' have three of the features....
so we have to remove them fromour annotations to match
"""
missing_columns = [
    'Features [Vision - Complexity]',
    'Features [Motor - Practice]',
    'Features [Causal - Caused]'
]
annot.drop(columns=missing_columns, inplace=True)
print(annot.shape)
annot

(18, 66)


,Timestamp,Email Address,SentenceID,SentenceText,Features [Vision - Vision],Features [Vision - Bright],Features [Vision - Dark],Features [Vision - Color],Features [Vision - Pattern],Features [Vision - Large],Features [Vision - Small],Features [Vision - Motion],Features [Vision - Biomotion],Features [Vision - Fast],Features [Vision - Slow],Features [Vision - Shape],Features [Vision - Face],Features [Vision - Body],Features [Somatic - Touch],Features [Somatic - Temperature],Features [Somatic - Texture],Features [Somatic - Weight],Features [Somatic - Pain],Features [Audition - Audition],Features [Audition - Loud],Features [Audition - Low],Features [Audition - High],Features [Audition - Sound],Features [Audition - Music],Features [Audition - Speech],Features [Gustation - Taste],Features [Olfaction - Smell],Features [Motor - Head],Features [Motor - UpperLimb],Features [Motor - LowerLimb],Features [Spatial - Landmark],Features [Spatial - Path],Features [Spatial - Scene],Features [Spatial - Near],Features [Spatial - Toward],Features [Spatial - Away],Features [Spatial - Number],Features [Temporal - Time],Features [Temporal - Duration],Features [Temporal - Long],Features [Temporal - Short],Features [Causal - Consequential],Features [Social - Social],Features [Social - Human],Features [Social - Communication],Features [Social - Self],Features [Cognition - Cognition],Features [Emotion - Benefit],Features [Emotion - Harm],Features [Emotion - Pleasant],Features [Emotion - Unpleasant],Features [Emotion - Happy],Features [Emotion - Sad],Features [Emotion - Angry],Features [Emotion - Disgusted],Features [Emotion - Fearful],Features [Emotion - Surprised],Features [Drive - Drive],Features [Drive - Needs],Features [Attention - Attention],Features [Attention - Arousal]
0,3/9/2022 17:22:31,katrin.erk@utexas.edu,4,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,1,0,3,3,0,0,1,3,3,0,1,6,0,6,3,3,3,0,6,0,0,0,0,0,0,0,0,3,0,3,3,3,5,1,5,4,4,0,0,0,4,0,6,0,5,4,3,4,0,6,0,6,0,6,3,1,6,4,4,6,4,4
1,3/21/2022 14:05:20,gabriellachronis@utexas.edu,4,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,1,0,4,3,0,0,4,1,0,0,3,3,0,5,0,1,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,3,1,1,0,0,6,0,1,0,0,1,0,6,0,6,0,2,2,4,6,4,2,2,0,3
2,3/10/2022 19:33:15,katrin.erk@utexas.edu,9,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.",5,0,0,2,0,3,3,4,4,3,3,6,3,6,0,0,0,3,5,1,0,0,0,2,0,0,0,0,2,3,6,3,4,6,3,5,0,0,3,3,1,2,5,3,6,1,0,4,0,5,0,3,0,3,0,1,3,0,0,0,4,1
3,3/21/2022 14:25:53,gabriellachronis@utexas.edu,9,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.",6,2,4,0,0,0,2,6,6,5,1,0,1,6,1,0,0,0,0,4,5,1,0,0,1,0,0,0,2,3,6,0,6,4,0,3,0,0,6,6,0,4,2,3,3,0,1,0,0,2,1,5,0,0,0,0,2,6,3,0,3,5
4,3/9/2022 20:21:36,katrin.erk@utexas.edu,13,"Over the past quarter century, the world has witnessed the fall of communism in Eastern Europe, the end of apartheid in South Africa, and the collapse of military dictatorships in Latin America.",0,0,0,0,0,2,0,2,2,3,0,0,3,3,0,0,0,0,2,3,3,0,0,0,1,3,0,0,1,1,1,0,0,0,0,0,0,0,0,3,3,0,6,6,6,6,1,2,6,3,6,3,6,3,5,5,5,5,5,1,6,6
5,3/21/2022 14:16:25,gabriellachronis@utexas.edu,13,"Over the past quarter century, the world has witnessed the fall of communism in Eastern Europe, the end of apartheid in South Africa, and the collapse of military dictatorships in Latin America.",3,0,0,0,0,3,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,6,0,1,0,0,5,3,2,0,6,6,2,0,0,5,3,4,0,4,3,3,4,5,0,0,4,0,6,1
6,3/10/2022 19:43:00,katrin.erk@utexas.edu,20,"She's committed to helping raise money to find a cure for breast canc

In [13]:
# get the gold vectors for the sentences in question

sentence_ids = annot['SentenceID'].unique()
sentence_ids

array([ 4,  9, 13, 20, 31, 34, 38, 39, 47])

In [14]:
gold_features = []

for sentence_id in sentence_ids:
    both_annotations = annot[annot['SentenceID']==sentence_id]
    #print(both_annotations)

    item = {
        "sentence_id": both_annotations.iloc[0]['SentenceID'],
        "cue_word": None,
        "context": both_annotations.iloc[0]['SentenceText'],
        #"vector": None
    }
    

    #print(both_annotations.columns)
    drop = both_annotations.drop(columns=["Timestamp", "Email Address", "SentenceID", "SentenceText"])

    vec = drop.mean(axis=0)
    vec = vec.to_numpy()  # output 4th row as Numpy array

    print(vec)

    item["vector"] = vec
    
    gold_features.append(item)    


    
gold_features = pd.DataFrame.from_records(gold_features)

words = ['collapse',
        'collapse',
        'collapse',
        'cure',
        'cure',
        'cure',
        'flow',
        'flow',
        'flow']

gold_features['cue_word'] = words

print(gold_features)

[1.  0.  3.5 3.  0.  0.  2.5 2.  1.5 0.  2.  4.5 0.  5.5 1.5 2.  2.  0.
 6.  0.  0.  0.  0.  0.  0.  0.  0.  1.5 0.  1.5 1.5 1.5 2.5 0.5 5.  2.
 2.  1.5 0.5 0.5 2.  0.  6.  0.  3.  2.  1.5 2.5 0.  6.  0.  6.  0.  4.
 2.5 2.5 6.  4.  3.  4.  2.  3.5]
[5.5 1.  2.  1.  0.  1.5 2.5 5.  5.  4.  2.  3.  2.  6.  0.5 0.  0.  1.5
 2.5 2.5 2.5 0.5 0.  1.  0.5 0.  0.  0.  2.  3.  6.  1.5 5.  5.  1.5 4.
 0.  0.  4.5 4.5 0.5 3.  3.5 3.  4.5 0.5 0.5 2.  0.  3.5 0.5 4.  0.  1.5
 0.  0.5 2.5 3.  1.5 0.  3.5 3. ]
[1.5 0.  0.  0.  0.  2.5 0.  1.  2.  1.5 1.5 0.  1.5 1.5 0.  0.  0.  0.
 1.  1.5 1.5 0.  0.  0.  0.5 3.  0.  0.  0.5 0.5 0.5 1.  0.  3.  0.  0.5
 0.  0.  2.5 3.  2.5 0.  6.  6.  4.  3.  0.5 3.5 4.5 3.5 3.  3.5 4.5 3.
 4.5 5.  2.5 2.5 4.5 0.5 6.  3.5]
[1.5 0.  0.  0.  0.  0.  1.5 0.5 1.  0.  0.  0.5 1.5 1.5 2.  0.  0.  0.
 4.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.5 1.  0.  0.  1.5 0.  2.
 0.  0.  5.  3.  3.  0.5 6.  5.5 4.5 3.  1.5 3.  6.  0.5 4.5 2.  6.  2.5
 1.5 0.  2.  3.5 3.  6.  3.5 3

In [15]:
"""
models and save paths

"""
model_save_paths = [
    '../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.binder.5k.300components.500max_iters',
    '../trained_models/model.plsr.binder.1k.300components.500max_iters',
    '../trained_models/model.modabs.binder.5k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
    '../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
]


glove_model_save_paths = [
    '../trained_models/model.ffnn.binder.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.binder.glove.300components.500max_iters',
    '../trained_models/model.modabs.binder.glove.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
]

In [16]:
# function ported from homonymous mcrae word evaluation and changed to use our data instead of stored gold vectors


"""
data is a list of tuples of (cue word, lemma, context)

for each model, we want to run an analysis of these
"""

def pluralize(string):
    if string == 'mouse':
        return 'mice'
    else:
        return string + 's'

def top_n_binder_features(gold_vector, n):
    features = [
'Vision',
'Bright',
'Dark',
'Color',
'Pattern',
'Large',
'Small',
'Motion',
'Biomotion',
'Fast',
'Slow',
'Shape',
'Face',
'Body',
'Touch',
'Temperature',
'Texture',
'Weight',
'Pain',
'Audition',
'Loud',
'Low',
'High',
'Sound',
'Music',
'Speech',
'Taste',
'Smell',
'Head',
'UpperLimb',
'LowerLimb',
'Landmark',
'Path',
'Scene',
'Near',
'Toward',
'Away',
'Number',
'Time',
'Duration',
'Long',
'Short',
'Consequential',
'Social',
'Human',
'Communication',
'Self',
'Cognition',
'Benefit',
'Harm',
'Pleasant',
'Unpleasant',
'Happy',
'Sad',
'Angry',
'Disgusted',
'Fearful',
'Surprised',
'Drive',
'Needs',
'Attention',
'Arousal'
]    
    vec_dict = dict()
    for i in range(0,len(gold_vector)):
        feat = features[i]
        val = gold_vector[i]
        vec_dict[feat] = val
    norm = {k: v for k, v in sorted(vec_dict.items(), key=lambda item: item[1])}
    # why the fuck did i have to cange this now when in the library code for top n it's :n ?
    # slightly worried about this and also weirdly hopeful. hopeful and dejected that maybe I've caused
    # myself so much pain when really maybe I might have been doing okay long ago
    top_n = list(norm)[-n:]
    top_n.reverse()
    feats = []

    return top_n
    #print(top_10)
        
    
def evaluate_in_context(model, df, bert, glove=False, model_name=None):
    """
    df has columns
    sentence_id
    word
    sentence
    features (numpy array)
    """
    
    n = 0
    
    cosines = []
    top_10_precs = []
    top_20_precs = []
    top_k_precs = []
    correlations = []
    rows = []

    num_top_10 = 0
    num_top_20 = 0
    num_total = 0

    for index, row in df.iterrows():
        cue_word = row.cue_word        
        lemma = row.cue_word
        context = row.context
        
#         print(cue_word)
#         print(lemma)
#         print(context)
        
        n +=1

        # predict features in context
        singular = lemma
        plural = pluralize(singular)
        try:
            prediction = model.predict_in_context(singular, context, bert, glove=glove)
            #top_k =  model.predict_top_n_features_in_context(singular, context, k, bert)
            word = singular
        except:
            prediction = model.predict_in_context(plural, context, bert, glove=glove)
            #top_k =  model.predict_top_n_features_in_context(plural, context, k, bert=bert)
            word = plural

        # get gold feature
        gold_vector = row.vector
        # gold_feats = model.feature_norms.get_features(cue_word)
        #k = len(gold_feats)

#         print(prediction.shape)
#         print(gold_vector.shape)
        # get cosines
        cos = 1 - cosine(prediction, gold_vector)
        cosines.append(cos)

        # get MAP at 10
        top_10 = model.predict_top_n_features_in_context(word, context, 10, vec=prediction, bert=bert)
        top_10_gold = top_n_binder_features(gold_vector, 10)
        
        num_in_top_10 = len(set(top_10).intersection(set(top_10_gold)))
        top_10_prec = num_in_top_10 / len(top_10_gold)
        top_10_precs.append(top_10_prec)

        # get MAP at 20
        top_20 = model.predict_top_n_features_in_context(word, context, 20, vec=prediction, bert=bert)
        top_20_gold = top_n_binder_features(gold_vector, 20)

        num_in_top_20 = len(set(top_20).intersection(set(top_20_gold)))
        top_20_prec = num_in_top_20 / len(top_20_gold)
        top_20_precs.append(top_20_prec)

        # get MAP at k
#         gold_len = len(gold_feats)
#         top_k = model.predict_top_n_features_in_context(word, context, gold_len, vec=prediction, bert=bert)
#         num_in_top_k = len(set(top_k).intersection(set(gold_feats)))
#         top_k_prec = num_in_top_k / gold_len
#         top_k_precs.append(top_k_prec)

        # get correlation
        corr, p = spearmanr(prediction, gold_vector)
        correlations.append(corr) 
    
        row = (cue_word, context, lemma, top_10_prec, top_20_prec, top_10, top_10_gold, cos)
        rows.append(row)
    
    cols = ['cue_word', 
               'context', 
               'lemma', 
               'top_10_prec', 
               'top_20_prec', 
               'top_10', 
               'top_10_gold', 
               'cos']
    
        
    avg_top_10_prec = np.average(top_10_precs)
    avg_top_20_prec = np.average(top_20_precs)
    average_correlation = np.average(correlations)
    average_cosine = np.average(cosines)
        
        
    print("Average cosine between gold and predicted feature norms: %s" % average_cosine)
    print("average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector: ", top_10_prec)
    print("average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector: ", top_20_prec)
    print("correlation between gold and predicted vectors: %s " % average_correlation)        
    print("total number of predictions: ", n)
    
    results_df = pd.DataFrame.from_records(rows, columns = cols)
    results_df['avg_top_10_prec'] = avg_top_10_prec
    results_df['avg_top_20_prec'] = avg_top_20_prec
    results_df['avg_correlation'] = average_correlation
    results_df['avg_cosine'] = average_cosine
    
    # TODO make this possible!!!!
    # results_df['model'] = model.name ()
    results_df['model'] = model_name

    
    return results_df
    


In [17]:
# then we take these sentences and run them through the model. which model? all of them?
scores_dfs = []




for save_path in model_save_paths:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")   
    model = torch.load(save_path)
 
    df = evaluate_in_context(model, gold_features, bert, glove=False, model_name=save_path)
    scores_dfs.append(df)


# need to do this separately because we dont have nice self examining attributes for our model classes to tell when its 
# been trained on glove
for save_path in glove_model_save_paths:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    
    model = torch.load(save_path)
 
    df = evaluate_in_context(model, gold_features, bert, glove=True, model_name=save_path)
    scores_dfs.append(df)

df = pd.concat(scores_dfs)
df.to_csv('../results/binder_gold_annotation_eval.csv')

****************************************
*** Evaluating ../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300 model ***
****************************************
Average cosine between gold and predicted feature norms: 0.763824782006668
average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector:  0.2
average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector:  0.45
correlation between gold and predicted vectors: 0.4917423461339603 
total number of predictions:  9
****************************************
*** Evaluating ../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300 model ***
****************************************
Average cosine between gold and predicted feature norms: 0.743156501358697
average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector:  0.2
average Percentage (%) of gold gol

In [59]:
pd.read_csv('../results/binder_gold_annotation_eval.csv', headers=True)

,Unnamed: 0,cue_word,context,lemma,top_10_prec,top_20_prec,top_10,top_10_gold,cos,avg_top_10_prec,avg_top_20_prec,avg_correlation,avg_cosine,model
0,0,collapse,Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.,collapse,0.6,0.45,"['Surprised', 'Fearful', 'Angry', 'Pain', 'Sad', 'Arousal', 'Harm', 'Attention', 'Consequential', 'Unpleasant']","['Fearful', 'Unpleasant', 'Harm', 'Consequential', 'Pain', 'Body', 'Near', 'Shape', 'Needs', 'Surprised']",0.778,0.411,0.350,4.917e-01,0.764,../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
1,1,collapse,"The chess master was slowly approaching on his sandpiper legs, and I could see that he was about to collapse under the weight of the garment bag he was carrying.",collapse,0.0,0.35,"['Short', 'Sad', 'Arousal', 'Pain', 'Fearful', 'Surprised', 'Unpleasant', 'Harm', 'Attention', 'Consequential']","['LowerLimb', 'Body', 'Vision', 'Scene', 'Path', 'Biomotion', 'Motion', 'Human', 'Duration', 'Time']",0.730,0.411,0.350,4.917e-01,0.764,../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
2,2,collapse,"Over the past quarter century, the world has witnessed the fall of communism in Eastern Europe, the end of apartheid in South Africa, and the collapse of military dictatorships in Latin America.",collapse,0.5,0.45,"['Sad', 'Social', 'Angry', 'Harm', 'Fearful', 'Surprised', 'Arousal', 'Consequential', 'Attention', 'Unpleasant']","['Attention', 'Social', 'Consequential', 'Disgusted', 'Drive', 'Angry', 'Happy', 'Benefit', 'Human', 'Arousal']",0.814,0.411,0.350,4.917e-01,0.764,../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
3,3,cure,"She's committed to helping raise money to find a cure for breast cancer, the disease that took her grandmother's life (Avon's donated $350 million and mammogram equipment to countries like Brazil).",cure,0.5,0.30,"['Drive', 'Smell', 'Happy', 'Needs', 'Taste', 'Scene', 'Pleasant', 'Consequential', 'Benefit', 'Vision']","['Needs', 'Happy', 'Benefit', 'Consequential', 'Social', 'Time', 'Pleasant', 'Human', 'Pain', 'Attention']",0.728,0.411,0.350,4.917e-01,0.764,../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
4,4,cure,There is a tendency to think that wealth is a cure for one's barbaric and primitive tendencies when it merely just mask it from the outside world.,cure,0.4,0.40,"['Communication', 'Attention', 'Needs', 'Pleasant', 'Arousal', 'Happy', 'Drive', 'Consequential', 'Social', 'Benefit']","['Social', 'Consequential', 'Human', 'Cognition', 'Communication', 'Long', 'Duration', 'Unpleasant', 'Harm', 'Benefit']",0.744,0.411,0.350,4.917e-01,0.764,../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
5,5,cure,The same system which creates poverty can not cure it.,cure,0.6,0.40,"['Cognition', 'Attention', 'Pleasant', 'Needs', 'Happy', 'Consequential', 'Drive', 'Social', 'Benefit', 'Arousal']","['Human', 'Social', 'Consequential', 'Drive', 'Unpleasant', 'Harm', 'Benefit', 'Attention', 'Needs', 'Angry']",0.826,0.411,0.350,4.917e-01,0.764,../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
6,6,flow,"Fingers exquisitely close to the body of the trout, Louisa felt the flow of water change.",flow,0.4,0.25,"['Self', 'Arousal', 'Fast', 'Motion', 'Benefit', 'Path', 'Consequential', 'Needs', 'Attention', 'Pleasant']","['Near', 'Touch', 'Path', 'Temperature', 'Motion', 'Pleasant', 'Scene', 'Fast', 'Sound', 'Audition']",0.717,0.411,0.350,4.917e-01,0.764,../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300
7,7,flow,"You had to be ready: Hundred-gallon drums could be filled with water in minutes, while flash floods were known to sweep everything away in rushing river homestead, swales dug into the ground helped direct the flow of water into drums and storage containers buried into the earth, creating reserv

In [60]:
m = torch.load('../trained_models/model.ffnn.binder.glove.50epochs.0.5dropout.lr1e-4.hsize300')
s1 = "Fingers exquisitely close to the body of the trout, Louisa felt the flow of water change."
s2 = "Her doctor identified her condition as uterine prolapse-a collapse of the uterus into the vagina that's symptomatic of damage or weakness in the pelvic muscles and ligaments.""

In [117]:
m.predict_top_n_features_in_context('collapse')

(62,)

In [118]:
m.feature_norms.get_features('doctor')

['Vision',
 'Bright',
 'Dark',
 'Color',
 'Pattern',
 'Large',
 'Small',
 'Motion',
 'Biomotion',
 'Fast',
 'Slow',
 'Shape',
 'Face',
 'Body',
 'Touch',
 'Temperature',
 'Texture',
 'Weight',
 'Pain',
 'Audition',
 'Loud',
 'Low',
 'High',
 'Sound',
 'Music',
 'Speech',
 'Taste',
 'Smell',
 'Head',
 'UpperLimb',
 'LowerLimb',
 'Landmark',
 'Path',
 'Scene',
 'Near',
 'Toward',
 'Away',
 'Number',
 'Time',
 'Duration',
 'Long',
 'Short',
 'Consequential',
 'Social',
 'Human',
 'Communication',
 'Self',
 'Cognition',
 'Benefit',
 'Harm',
 'Pleasant',
 'Unpleasant',
 'Happy',
 'Sad',
 'Angry',
 'Disgusted',
 'Fearful',
 'Surprised',
 'Drive',
 'Needs',
 'Attention',
 'Arousal']

In [14]:
# get just the word thats used in binder feature dataset as the label
for col in drop.columns:
    word = col.split()[-1][:-1]
    print("'"+ word + "',")

'Vision',
'Bright',
'Dark',
'Color',
'Pattern',
'Large',
'Small',
'Motion',
'Biomotion',
'Fast',
'Slow',
'Shape',
'Face',
'Body',
'Touch',
'Temperature',
'Texture',
'Weight',
'Pain',
'Audition',
'Loud',
'Low',
'High',
'Sound',
'Music',
'Speech',
'Taste',
'Smell',
'Head',
'UpperLimb',
'LowerLimb',
'Landmark',
'Path',
'Scene',
'Near',
'Toward',
'Away',
'Number',
'Time',
'Duration',
'Long',
'Short',
'Consequential',
'Social',
'Human',
'Communication',
'Self',
'Cognition',
'Benefit',
'Harm',
'Pleasant',
'Unpleasant',
'Happy',
'Sad',
'Angry',
'Disgusted',
'Fearful',
'Surprised',
'Drive',
'Needs',
'Attention',
'Arousal',


In [143]:
for col in drop.columns:
    print("'"+ col + "',")

'Features [Vision - Vision]',
'Features [Vision - Bright]',
'Features [Vision - Dark]',
'Features [Vision - Color]',
'Features [Vision - Pattern]',
'Features [Vision - Large]',
'Features [Vision - Small]',
'Features [Vision - Motion]',
'Features [Vision - Biomotion]',
'Features [Vision - Fast]',
'Features [Vision - Slow]',
'Features [Vision - Shape]',
'Features [Vision - Face]',
'Features [Vision - Body]',
'Features [Somatic - Touch]',
'Features [Somatic - Temperature]',
'Features [Somatic - Texture]',
'Features [Somatic - Weight]',
'Features [Somatic - Pain]',
'Features [Audition - Audition]',
'Features [Audition - Loud]',
'Features [Audition - Low]',
'Features [Audition - High]',
'Features [Audition - Sound]',
'Features [Audition - Music]',
'Features [Audition - Speech]',
'Features [Gustation - Taste]',
'Features [Olfaction - Smell]',
'Features [Motor - Head]',
'Features [Motor - UpperLimb]',
'Features [Motor - LowerLimb]',
'Features [Spatial - Landmark]',
'Features [Spatial - Path]'

SyntaxError: invalid syntax (3904491136.py, line 1)

In [41]:
jfk = gold_features.iloc[0].vector

In [42]:
efg = ['Vision',
'Bright',
'Dark',
'Color',
'Pattern',
'Large',
'Small',
'Motion',
'Biomotion',
'Fast',
'Slow',
'Shape',
'Face',
'Body',
'Touch',
'Temperature',
'Texture',
'Weight',
'Pain',
'Audition',
'Loud',
'Low',
'High',
'Sound',
'Music',
'Speech',
'Taste',
'Smell',
'Head',
'UpperLimb',
'LowerLimb',
'Landmark',
'Path',
'Scene',
'Near',
'Toward',
'Away',
'Number',
'Time',
'Duration',
'Long',
'Short',
'Consequential',
'Social',
'Human',
'Communication',
'Self',
'Cognition',
'Benefit',
'Harm',
'Pleasant',
'Unpleasant',
'Happy',
'Sad',
'Angry',
'Disgusted',
'Fearful',
'Surprised',
'Drive',
'Needs',
'Attention',
'Arousal']

In [44]:
list(zip(efg,jfk))

[('Vision', 1.0),
 ('Bright', 0.0),
 ('Dark', 3.5),
 ('Color', 3.0),
 ('Pattern', 0.0),
 ('Large', 0.0),
 ('Small', 2.5),
 ('Motion', 2.0),
 ('Biomotion', 1.5),
 ('Fast', 0.0),
 ('Slow', 2.0),
 ('Shape', 4.5),
 ('Face', 0.0),
 ('Body', 5.5),
 ('Touch', 1.5),
 ('Temperature', 2.0),
 ('Texture', 2.0),
 ('Weight', 0.0),
 ('Pain', 6.0),
 ('Audition', 0.0),
 ('Loud', 0.0),
 ('Low', 0.0),
 ('High', 0.0),
 ('Sound', 0.0),
 ('Music', 0.0),
 ('Speech', 0.0),
 ('Taste', 0.0),
 ('Smell', 1.5),
 ('Head', 0.0),
 ('UpperLimb', 1.5),
 ('LowerLimb', 1.5),
 ('Landmark', 1.5),
 ('Path', 2.5),
 ('Scene', 0.5),
 ('Near', 5.0),
 ('Toward', 2.0),
 ('Away', 2.0),
 ('Number', 1.5),
 ('Time', 0.5),
 ('Duration', 0.5),
 ('Long', 2.0),
 ('Short', 0.0),
 ('Consequential', 6.0),
 ('Social', 0.0),
 ('Human', 3.0),
 ('Communication', 2.0),
 ('Self', 1.5),
 ('Cognition', 2.5),
 ('Benefit', 0.0),
 ('Harm', 6.0),
 ('Pleasant', 0.0),
 ('Unpleasant', 6.0),
 ('Happy', 0.0),
 ('Sad', 4.0),
 ('Angry', 2.5),
 ('Disgusted', 2